In [10]:
# DO THIS FIRST: FROM THE MENU AT THE TOP, CLICK' 'CELL' THEN 'RUN ALL'.

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import binom
from ipywidgets import interact, fixed, interact_manual

In [11]:
# structure functions --- go wild!

def horizontal(numDay):
    """n agents each bet once."""
    return [1]*numDay

def vertical(numDay):
    """One agent betting n times."""
    return [numDay]

def nByK(k):
    """k agents each betting n times."""
    return lambda numDay : [numDay]*k

def kByN(k):
    """n agents each betting k times."""
    return lambda numDay : [k]*numDay

def nByN(numDay):
    """n agents each betting n times."""
    return [numDay]*numDay

structuresToName = {'horizontal' : horizontal,
                    'vertical' : vertical,
                    'n by n' : nByN}

structuresToDescription = dict([v,k] for k,v in structuresToName.items())

In [12]:
def bankrolls(betStructure,numDay,numWinsList,odds,betProp,initialBankroll=1):
    """Computes agents' bankrolls at end of Day n, given win counts and betting structure."""
    numBetsList = betStructure(numDay)
    return np.array([initialBankroll * (1+odds*betProp)**numWins * (1-betProp)**(numBets - numWins) \
            for (numWins,numBets) in zip(numWinsList,numBetsList)])

In [13]:
# performance measures

def averageGrowthRate(bankrolls,betStructure,numDay,initialBankroll=1):
    """Computes each agent's growth rate, then averages."""
    return np.mean((bankrolls / initialBankroll) ** (1 / np.array(betStructure(numDay))))

def growthRateOfAverage(bankrolls,betStructure,numDay,initialBankroll=1):
    """Computes agents' average bankroll, then finds growth rate."""
    return np.mean(bankrolls)**(1 / betStructure(numDay)[0]) # what if people have bet different numbers of times?

measuresToName = {'average growth rate' : averageGrowthRate,
                 'growth rate of average' : growthRateOfAverage}

measuresToDescription = dict([v,k] for k,v in measuresToName.items())

In [14]:
def scaledBankrolls(betStructure,numDay,measure,numWinsList,odds,betProp,initialBankroll=1):
    """Computes performance measure of agents' bankrolls at end of Day n, given win counts and betting structure."""
    return measure(bankrolls(betStructure,numDay,numWinsList,odds,betProp,initialBankroll),
                       betStructure,numDay,initialBankroll)

In [15]:
def simulateBetting(betStructure,numDay,measure,odds,winProb,betProp,initialBankroll=1):
    """Shows how agents' performance evolves over days."""
    performance = []
    cumWins = np.array([np.cumsum(np.random.random(numBets) < winProb) for numBets in betStructure(numDay)]).T
    print(cumWins)
    for day in range(1,numDay+1):
        numWinsList = [cumWins[numBets-1,index] for (index,numBets) in enumerate(betStructure(day))]
        print(numWinsList)
        print(bankrolls(betStructure,day,numWinsList,odds,betProp))
        performance += [scaledBankrolls(betStructure,day,measure,numWinsList,odds,betProp,initialBankroll)]
        return performance

simulateBetting(kByN(3),5,averageGrowthRate,2,.5,.5)

[[0 1 0 1 0]
 [0 2 0 1 1]
 [0 3 1 1 2]]
[0]
[0.125]


[0.5]

In [16]:
def plotFourPerformances(betStructure,measure,numDay,odds,winProb,
                         betProp0=0.,betProp1=.4,betProp2=.8,betProp3=1.,initialBankroll=1):
    cumWins = np.array([np.cumsum(np.random.random(numBets) < winProb) for numBets in betStructure(numDay)]).T
    betProps = [betProp0,betProp1,betProp2,betProp3]
    for betProp in betProps:
        
        # find how that bet proportion does
        performance = []
        for day in range(1,numDay+1):
            numWinsList = [cumWins[numBets-1,index] for (index,numBets) in enumerate(betStructure(day))]
            performance += [scaledBankrolls(betStructure,day,measure,numWinsList,odds,betProp,initialBankroll)]
        
        # plot its performance
        label = 'bet proportion: {}'.format(betProp)
        plt.plot(range(1,numDay+1),performance,label=label)
    
    plt.xlabel('number of days'); plt.ylabel(measuresToDescription[measure])
    plt.title("""Plots how the agents perform over time 
    when the betting structure is {}.""".format(structuresToDescription[betStructure]))
    plt.legend()
    plt.show()
    
interact_manual(plotFourPerformances, betStructure=structuresToName, measure=measuresToName,
         odds=(0,10,0.5), numDay=(1,100), winProb=(0,1.), betProp0=(0,1,0.05), betProp1=(0,1,0.05),
         betProp2=(0,1,0.05), betProp3=(0,1,0.05), initialBankroll=fixed(1))

interactive(children=(Dropdown(description='betStructure', options={'horizontal': <function horizontal at 0x00…

<function __main__.plotFourPerformances(betStructure, measure, numDay, odds, winProb, betProp0=0.0, betProp1=0.4, betProp2=0.8, betProp3=1.0, initialBankroll=1)>

In an *n by 2 gambling problem*, two people, $A_1$ and $A_2$, face gambling problems with win probability $p$ and odds $b$:1 each day. Each of their bankrolls before betting on Day 1 is $r$. And their bankrolls roll over: each persons' bankroll before betting on Day $k+1$ is whatever her bankroll after betting is on Day $k$. They have to bet the same proportion of their bankrolls each day, and as each other. The outcomes of their gambles are independent.

Take Example 1 above, where $p=\frac{1}{2}$, $b=2$ and $r$ is 100 dollars. Suppose each day they both bet proportion $m$, for some $m \in [0,1]$. What will happen? Here's one way things might go. On Day 1, $A_1$ bets and wins and $A_2$ bets and loses, say. $A_1$ now has $100(1+bm)$ dollars and $A_2$ has $100(1-m)$ dollars. Their combined bankroll is $100(2+(b-1)m)$. On Day 2, both bet and lose, say. $A_1$ now has $100(1+bm)(1-m)$ dollars and $A_2$ now has $100(1-m)^2$ dollars. Their combined bankroll is $100(1-m)(2+(b-1)m)$ dollars. And so on. We're interested in their combined bankroll as they bet more and more times. 

Let's proceed indirectly, by looking not at their combined bankroll as they bet more and more times, but at their *average growth rate*. What's that? Well, if their final bankrolls after betting $n$ times with initial bankroll $r$ are $f_1$ and $f_2$, then their average growth rate is $g = \frac{1}{2} \big( \sqrt[n]{\frac{f_1}{r}} + \sqrt[n]{\frac{f_2}{r}} \big)$.

Each time one of them wins, she multiplies her bankroll by $(1+bm)$ and each time one of them loses, she multiplies her bankroll by $(1-m)$. So if their initial bankroll is $r$ dollars, they bet $n$ times each, $A_1$ wins $k_1$ times, and $A_2$ wins $k_2$ times, their combined bankroll is:

$$ r \cdot \Big( (1+bm)^{k_1} \cdot (1-m)^{n-k_1} + (1+bm)^{k_2} \cdot (1-m)^{n-k_2} \Big) $$

And their average growth rate is:

$$ \frac{1}{2} \Big( (1+bm)^{\frac{k_1}{n}} \cdot (1-m)^{\frac{n-k_1}{n}} + (1+bm)^{\frac{k_2}{n}} \cdot (1-m)^{\frac{n-k_2}{n}} \Big) $$

You can evaluate these two expressions, their combined bankroll and their average growth rate after $k_1$ and $k_2$ wins respectively in $n$ bets, for different values for $n$, $k_1$, $k_2$, $b$, and $m$ using the sliders below. When calculating their combined bankroll, we assume their initial bankroll $r$ is 100 dollars.

In [17]:
def nBy2CombinedBankrolls(n,k1,k2,odds,betProp,initialBankroll=1):
    """Finds combined bankroll after k1, k2 wins in n vertical rounds."""
    return initialBankroll * ((1+odds*betProp)**k1 * (1-betProp)**(n-k1) +
                              (1+odds*betProp)**k2 * (1-betProp)**(n-k2))

def nBy2AverageGrowthRate(n,k1,k2,odds,betProp):
    """Finds average growth rate after k1, k2 wins in n vertical rounds."""
    return 0.5 * (((1+odds*betProp)**k1 * (1-betProp)**(n-k1))**(1/n) +
                  ((1+odds*betProp)**k2 * (1-betProp)**(n-k2))**(1/n))

def displayNBy2Stuff(n=50,k1=30,k2=10,odds=2,betProp=.5):
    """Returns bankroll info after k1, k2 wins in n vertical rounds, assuming initial bankroll is 100 dollars."""
    if k1 > n or k2 > n:
        print("That doesn't make sense: number of wins > number of bets!")
    else:
        combined = nBy2CombinedBankrolls(n,k1,k2,odds,betProp,100)
        roundedCombined = int(np.around(combined))
        averageGrowthRate = nBy2AverageGrowthRate(n,k1,k2,odds,betProp)
        roundedAverageGrowthRate = np.around(averageGrowthRate,2)
        print('Combined bankroll is {} dollars (to nearest dollar).'.format(roundedCombined))
        print('Average growth rate is {} (to 2 decimal places).'.format(roundedAverageGrowthRate))
        
interact(displayNBy2Stuff,n=(1,100),k1=(0,100),k2=(0,100),odds=(0,10,0.5),betProp=(0,1,0.05))

interactive(children=(IntSlider(value=50, description='n', min=1), IntSlider(value=30, description='k1'), IntS…

<function __main__.displayNBy2Stuff(n=50, k1=30, k2=10, odds=2, betProp=0.5)>

How their average growth rate changes as they bet more and more times is a chancy matter, since the outcome of each bet is chancy. You can plot how things might go for a few different choices of bet proportion using the sliders below. Whenever you move a slider, a fresh sequence of bets is simulated and it plots the average growth rate against the number of bets for each choice of bet proportion.

In [18]:
def plotFourNBy2AverageGrowthRates(odds,winProbability,numRounds,betProp0=0.,betProp1=.4,
                             betProp2=.8,betProp3=1.):
    """Plots simulation of four average growth rates after n rounds for given bet proportions."""
    cumWins1 = np.cumsum(np.random.random(numRounds) < winProbability)
    cumWins2 = np.cumsum(np.random.random(numRounds) < winProbability)
    betProps = [betProp0,betProp1,betProp2,betProp3]
    for betProp in betProps:
        averageGrowthRates = [nBy2AverageGrowthRate(numRounds,numWins[0],numWins[1],odds,betProp) \
         for (numRounds,numWins) in enumerate(zip(cumWins1,cumWins2),start=1)]
        label = 'bet proportion: {}'.format(betProp)
        plt.plot(range(1,numRounds+1),averageGrowthRates,label=label)
    plt.xlabel("number of times they've each bet"); plt.ylabel('average growth rate')
    plt.legend()
    plt.show()
    
    
interact(plotFourNBy2AverageGrowthRates,odds=(0,10,0.5),winProbability=(0,1.), numRounds=(1,100),
         betProp0=(0,1,0.05),betProp1=(0,1,0.05),betProp2=(0,1,0.05),betProp3=(0,1,0.05))

interactive(children=(FloatSlider(value=5.0, description='odds', max=10.0, step=0.5), FloatSlider(value=0.5, d…

<function __main__.plotFourNBy2AverageGrowthRates(odds, winProbability, numRounds, betProp0=0.0, betProp1=0.4, betProp2=0.8, betProp3=1.0)>

In [19]:
def nBy3CombinedBankrolls(n,k1,k2,k3,odds,betProp,initialBankroll=1):
    """Finds combined bankroll after k1, k2, k3 wins in n vertical rounds."""
    return initialBankroll * ((1+odds*betProp)**k1 * (1-betProp)**(n-k1) +
                              (1+odds*betProp)**k2 * (1-betProp)**(n-k2) +
                              (1+odds*betProp)**k3 * (1-betProp)**(n-k3))

def nBy3AverageGrowthRate(n,k1,k2,k3,odds,betProp):
    """Finds average growth rate after k1, k2, k3 wins in n vertical rounds."""
    return 1/3 * (((1+odds*betProp)**k1 * (1-betProp)**(n-k1))**(1/n) +
                  ((1+odds*betProp)**k2 * (1-betProp)**(n-k2))**(1/n) +
                  ((1+odds*betProp)**k3 * (1-betProp)**(n-k3))**(1/n))

def displayNBy3Stuff(n=50,k1=30,k2=20,k3=10,odds=2,betProp=.5):
    """Returns bankroll info after k1, k2, k3 wins in n vertical rounds, assuming initial bankroll is 100 dollars."""
    if max(k1,k2,k3) > n:
        print("That doesn't make sense: number of wins > number of bets!")
    else:
        combined = nBy3CombinedBankrolls(n,k1,k2,k3,odds,betProp,100)
        roundedCombined = int(np.around(combined))
        averageGrowthRate = nBy3AverageGrowthRate(n,k1,k2,k3,odds,betProp)
        roundedAverageGrowthRate = np.around(averageGrowthRate,2)
        print('Combined bankroll is {} dollars (to nearest dollar).'.format(roundedCombined))
        print('Average growth rate is {} (to 2 decimal places).'.format(roundedAverageGrowthRate))
        
interact(displayNBy3Stuff,n=(1,100),k1=(0,100),k2=(0,100),k3=(0,100),odds=(0,10,0.5),betProp=(0,1,0.05))

interactive(children=(IntSlider(value=50, description='n', min=1), IntSlider(value=30, description='k1'), IntS…

<function __main__.displayNBy3Stuff(n=50, k1=30, k2=20, k3=10, odds=2, betProp=0.5)>

In [20]:
def plotFourNBy3AverageGrowthRates(odds,winProbability,numRounds,betProp0=0.,betProp1=.4,
                             betProp2=.8,betProp3=1.):
    """Plots simulation of four average growth rates after n rounds for given bet proportions."""
    cumWins1 = np.cumsum(np.random.random(numRounds) < winProbability) # TBD improve this!
    cumWins2 = np.cumsum(np.random.random(numRounds) < winProbability)
    cumWins3 = np.cumsum(np.random.random(numRounds) < winProbability)
    betProps = [betProp0,betProp1,betProp2,betProp3]
    for betProp in betProps:
        averageGrowthRates = [nBy3AverageGrowthRate(numRounds,numWins[0],numWins[1],numWins[2],odds,betProp) \
         for (numRounds,numWins) in enumerate(zip(cumWins1,cumWins2,cumWins3),start=1)]
        label = 'bet proportion: {}'.format(betProp)
        plt.plot(range(1,numRounds+1),averageGrowthRates,label=label)
    plt.xlabel("number of times they've each bet"); plt.ylabel('average growth rate')
    plt.legend()
    plt.show()

interact(plotFourNBy3AverageGrowthRates,odds=(0,10,0.5),winProbability=(0,1.),numRounds=(1,100),
         betProp0=(0,1,0.05),betProp1=(0,1,0.05),betProp2=(0,1,0.05),betProp3=(0,1,0.05))

interactive(children=(FloatSlider(value=5.0, description='odds', max=10.0, step=0.5), FloatSlider(value=0.5, d…

<function __main__.plotFourNBy3AverageGrowthRates(odds, winProbability, numRounds, betProp0=0.0, betProp1=0.4, betProp2=0.8, betProp3=1.0)>

As in 2.1, except now each day an extra agent comes on the scene and catches up with the gambles.

Day 1. $A_1$ bets once.

Day 2. $A_1$ bets once, $A_2$ appears and bets twice, so each has bet twice total.

Day 3. $A_1$ bets once, $A_2$ bets once, $A_3$ appears and bets three times, so each has bet three times total.
And so on.

The number of agents involved increases with the number of days: at the end of Day $n$, $n$ agents have bet $n$ times each in independent vertical gambling problems.

Again, we look at average growth rate.

In [21]:
def nByNCombinedBankrolls(n,numWinsList,odds,betProp,initialBankroll=1):
    """Finds combined bankroll after n agents bet in n vertical rounds."""
    return np.array([(1+odds*betProp)**numWins * (1-betProp)**(n-numWins) for numWins in numWinsList]).sum()

def nByNAverageGrowthRate(n,numWinsList,odds,betProp):
    """Finds average growth rate after n agents bet in n vertical rounds."""
    return 1/n * np.array([((1+odds*betProp)**numWins * (1-betProp)**(n-numWins))**(1/n) for numWins in numWinsList]).sum()

def plotFourNByNAverageGrowthRates(odds,winProbability,numRounds,betProp0=0.,betProp1=.4,
                             betProp2=.8,betProp3=1.):
    """Plots simulation of four average growth rates after n rounds for given bet proportions."""
    cumWins = np.array([np.cumsum(np.random.random(numRounds) < winProbability) for i in range(numRounds)])
    betProps = [betProp0,betProp1,betProp2,betProp3]
    for betProp in betProps:
        averageGrowthRates = [nByNAverageGrowthRate(dayNumber+1,cumWins[0:dayNumber+1,dayNumber],odds,betProp) \
         for dayNumber in range(numRounds)]
        label = 'bet proportion: {}'.format(betProp)
        plt.plot(range(1,numRounds+1),averageGrowthRates,label=label)
    plt.xlabel("number of days"); plt.ylabel('average growth rate')
    plt.legend()
    plt.show()

interact(plotFourNByNAverageGrowthRates,odds=(0,10,0.5),winProbability=(0,1.),numRounds=(1,100),
         betProp0=(0,1,0.05),betProp1=(0,1,0.05),betProp2=(0,1,0.05),betProp3=(0,1,0.05))

interactive(children=(FloatSlider(value=5.0, description='odds', max=10.0, step=0.5), FloatSlider(value=0.5, d…

<function __main__.plotFourNByNAverageGrowthRates(odds, winProbability, numRounds, betProp0=0.0, betProp1=0.4, betProp2=0.8, betProp3=1.0)>